In [19]:
import codecs
import jieba as jieba
import jieba.analyse
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
response_sents = [];
#中文分局
def SentsTokenizer4Ch(text):
    sentences = re.split('(\r\n)',text)
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        sent = sent.replace('\r\n','')
        sent = sent.strip()
        if(i % 2 == 0):
            new_sents.append(sent)
        else:
            response_sents.append(sent)
         
    return new_sents
 
    
    
#删除所有符号,只保留字母、数字和中文
def remove_punctuation(line):
    #line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
#中文停用词
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  


In [21]:


myfile = "./data/qa.txt"
text = codecs.open(myfile, "r", "utf-8").read()
sent_tokens = SentsTokenizer4Ch(text)
sent_tokens[:30]

['\ufeff充电宝如何归还？',
 '能否租借多个充电宝？',
 '借出的充电宝是坏的怎么办？',
 '充电宝不能充电怎么办？',
 '归还充电宝后没有结束订单怎么办？',
 '扫码付钱后没有弹出充电宝怎么办？',
 '借出得充电宝不能充电怎么办？',
 '充电宝TYPE-C头不能充电',
 '充电宝USB头不能充电',
 '由于疫情管控,不能及时归还？']

In [22]:
#加载自定义词典
jieba.load_userdict("./data/dict.txt")
 
#加载停用词
stopwords = stopwordslist("./data/stop_words.txt")
# jieba.analyse.set_stop_words("./ntpl/stop_words.txt")

#删除除字母,数字，汉字以外的所有符号
df = pd.DataFrame(sent_tokens, columns=['sent'])
df['clean_set']=  df['sent'].apply(remove_punctuation)

#分词，并过滤停用词
df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jieba.cut(x)) if w not in stopwords]))
# df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jieba.cut(x))]))
df

,sent,clean_set,cut_sent
0,﻿充电宝如何归还？,充电宝如何归还,充电宝 归还
1,能否租借多个充电宝？,能否租借多个充电宝,能否 租借 多个 充电宝
2,借出的充电宝是坏的怎么办？,借出的充电宝是坏的怎么办,借出 充电宝 坏
3,充电宝不能充电怎么办？,充电宝不能充电怎么办,充电宝 不能 充电
4,归还充电宝后没有结束订单怎么办？,归还充电宝后没有结束订单怎么办,归还 充电宝 没有 结束订单
5,扫码付钱后没有弹出充电宝怎么办？,扫码付钱后没有弹出充电宝怎么办,扫码 付钱 没有 弹出 充电宝
6,借出得充电宝不能充电怎么办？,借出得充电宝不能充电怎么办,借出 得 充电宝 不能 充电
7,充电宝TYPE-C头不能充电,充电宝TYPEC头不能充电,充电宝 TYPEC头 不能 充电
8,充电宝USB头不能充电,充电宝USB头不能充电,充电宝 USB头 不能 充电
9,"由于疫情管控,不能及时归还？",由于疫情管控不能及时归还,由于 疫情 管控 不能 及时 归还


In [23]:
def response(user_response):
    robo_response=''
    user_response  = remove_punctuation(user_response)
    user_response= " ".join([w for w in list(jieba.cut(user_response)) if w not in stopwords])
    cut_sent = df.cut_sent.values.tolist()
    cut_sent.append(user_response)
    tfidf = TfidfVectorizer()

    tfidf_vec = tfidf.fit_transform(cut_sent)

    cos_sim = cosine_similarity(tfidf_vec[-1], tfidf_vec)
    idx=cos_sim.argsort()[0][-2]
    flat = cos_sim.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if(req_tfidf==0):
        robo_response=robo_response+"对不起,我不理解您的意思,我还是个小白...!"
        return(robo_response)
    else:
        #robo_response = robo_response+df.sent.values[idx]
        print(robo_response+df.sent.values[idx])
        robo_response = response_sents[idx]
        return(robo_response)

In [24]:

GREETING_INPUTS = ("你好", "hi", "有人", "在吗", "嗨","在不在","有人吗",'在','有人')
GREETING_RESPONSES = ["你好",  "我在", "请问,有什么可以帮您的吗?", "你好，我在", "你好，很高兴见到你！"]

def greeting(sentence):
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    text = rule.sub('',sentence)
    if text in GREETING_INPUTS:
        return random.choice(GREETING_RESPONSES)
    
    wordlist =  [w for w in jieba.cut(sentence)]
    for word in wordlist:
        if word in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


In [27]:
greeting("你好")

'请问,有什么可以帮您的吗?'

In [25]:

# 此函数用于返回查询结果
response("我的订单没有结束")

归还充电宝后没有结束订单怎么办？


'订单将会在您归还充电宝后10分钟内自动结束，如超过10分钟未结束，请联系人工客服。'

In [17]:
python 

NameError: name 'python' is not defined